<img src="https://hilpisch.com/tpq_logo_bic.png"
     width="30%"
     align="right"
     style="border-radius: 8px;">

# Derivatives Analytics with Python
**&mdash;Part II: Arbitrage Pricing**

&copy; Dr. Yves J. Hilpisch | The Python Quants

<a href="https://tpq.io" target="_blank">tpq.io</a> | <a href="https://linktr.ee/dyjh" target="_blank">linktr.ee/dyjh</a>

<img src="https://hilpisch.com/dawp_cover_small.png" width=30% align=left>

## Part II &mdash; Arbitrage Pricing

### Chapter 4 &mdash; Risk-Neutral Valuation

This notebook develops the core ideas of arbitrage-free pricing and risk-neutral valuation in
discrete-time models and prepares the ground for more general continuous-time frameworks.

The focus in Chapter 4 is on one-period and multi-period binomial models, replication, and
risk-neutral probabilities as tools for understanding how derivative prices relate to underlying
assets and the risk-free rate.

Reference notebooks and scripts for the book are available under `x_store/dawp/python36`.

The notebook is designed to run smoothly on Google Colab. A local setup is optional and
can be based on `python -m venv .venv`.

### 1. Environment check

We start by importing the core libraries for this class and by printing version
information to confirm that a modern Python environment is active.

In [ ]:
import sys  # access basic runtime information
from pathlib import Path  # path handling for optional figure export

import math  # elementary math functions

import numpy as np  # numerical arrays and linear algebra tools
import pandas as pd  # tabular data handling
import matplotlib as mpl  # matplotlib configuration
import matplotlib.pyplot as plt  # plotting

np.set_printoptions(precision=6, suppress=True)  # compact numeric output

print(sys.version.split()[0])  # Python version string
print("NumPy:", np.__version__)  # NumPy version string

FIG_SAVE = True  # set to True to export figures as PDFs
FIG_DIR = Path("../figures")  # figure output directory
FIG_DPI = 300  # target resolution for exported figures
FIG_DISPLAY = "svg"  # inline display format: "svg" or "png"

plt.style.use("seaborn-v0_8")  # readable plotting defaults
mpl.rcParams["figure.figsize"] = (8.0, 4.5)  # consistent figure size
mpl.rcParams["axes.grid"] = True  # show a grid for readability
mpl.rcParams["savefig.dpi"] = FIG_DPI  # default export resolution

try:
    from matplotlib_inline.backend_inline import (  # Jupyter helper
        set_matplotlib_formats,
    )
    set_matplotlib_formats(FIG_DISPLAY)  # configure inline plot rendering
except Exception:
    pass

if FIG_DISPLAY == "png":
    mpl.rcParams["figure.dpi"] = FIG_DPI  # high-resolution inline rendering


def maybe_save(fig, filename):
    # Optionally saves a Matplotlib figure as a PDF file.
    if not FIG_SAVE:
        return
    FIG_DIR.mkdir(parents=True, exist_ok=True)
    path = FIG_DIR / f"{filename}.pdf"
    fig.savefig(path, format="pdf", dpi=FIG_DPI)
    print(f"saved: {path}")

### 2. Discrete-time uncertainty

We formalize uncertainty in discrete time via a probability space and an information
structure. This provides the language needed for martingales and risk-neutral valuation.

- The probability space is $(\Omega, \mathcal{F}, \mathbb{P})$.
- Trading dates are $t \in \{0, 1, \dots, T\}$.
- Information is modeled by a filtration $(\mathcal{F}_t)_{t=0,\dots,T}$ with
  $\mathcal{F}_0 \subseteq \cdots \subseteq \mathcal{F}_T \subseteq \mathcal{F}$.
- A process $(X_t)$ is adapted if $X_t$ is $\mathcal{F}_t$-measurable.
- A martingale $(M_t)$ satisfies $\mathbb{E}[M_{t+1} \mid \mathcal{F}_t] = M_t$.

### 3. Discrete market model

A discrete market model specifies tradable assets, admissible trading strategies, and the
self-financing condition. The model uses a numéraire, typically the money-market account.

- Asset prices are given by $S_t = (S_t^{0}, S_t^{1}, \dots, S_t^{d})$.
- The numéraire $S_t^{0} > 0$ is often written as $B_t$ with $B_0 = 1$.
- A trading strategy is a predictable process $\varphi_t = (\varphi_t^{0}, \dots, \varphi_t^{d})$.
- Portfolio value is $V_t(\varphi) = \varphi_t \cdot S_t$.
- Self-financing means $V_{t+1}(\varphi) - V_t(\varphi) = \varphi_t \cdot (S_{t+1} - S_t)$.
- Discounted prices are $\tilde{S}_t = S_t / S_t^{0}$ and simplify martingale statements.
- A contingent claim is an $\mathcal{F}_T$-measurable payoff $H_T$.
- No-arbitrage rules out self-financing strategies with $V_0 \le 0$ and $V_T \ge 0$ a.s.
  with strict positivity occurring with positive probability.

### 4. One-period risk-neutral valuation

We now study a single-period binomial model for an equity underlying and a European call
option. The example illustrates how risk-neutral probabilities and discounted expected
payoffs translate the no-arbitrage principle into a numerical valuation.

In [ ]:
S0 = 100.0  # current underlying level
Su = 120.0  # up move in one period
Sd = 85.0  # down move in one period
r = 0.05  # risk-free rate per period
K = 100.0  # strike level of the call option

q = ((1.0 + r) * S0 - Sd) / (Su - Sd)  # risk-neutral probability
print(f"risk-neutral probability q = {q:.4f}")

Cu = max(Su - K, 0.0)  # call payoff in the up state
Cd = max(Sd - K, 0.0)  # call payoff in the down state
disc = 1.0 / (1.0 + r)  # discount factor for one period
C0 = disc * (q * Cu + (1.0 - q) * Cd)  # call present value

print(f"arbitrage-free call value C0 = {C0:.4f}")

### 5. Multi-period binomial tree

We now extend the one-period model to a multi-period binomial tree. The underlying price
can move up or down in each period, and a self-financing strategy replicates the option
payoff by dynamically adjusting the number of shares and the position in the risk-free asset.

In [ ]:
n_steps = 30  # number of time steps in the tree
T = 1.0  # maturity in years
r = 0.05  # annual risk-free rate
sigma = 0.20  # volatility parameter (per year)
S0 = 100.0  # initial underlying level
K = 100.0  # strike level of the call option

# per-step parameters in a CRR-style binomial tree
dt = T / n_steps  # length of one time step
u = np.exp(sigma * np.sqrt(dt))  # up factor per step

d = np.exp(-sigma * np.sqrt(dt))  # down factor per step
R_step = np.exp(r * dt)  # per-step gross risk-free return

# risk-neutral probability and discount factor per step
q = (R_step - d) / (u - d)
print(f"risk-neutral probability per step q = {q:.4f}")

dt_disc = 1.0 / R_step  # discount factor per step

# build terminal stock prices and option payoffs
S_T = np.array([
    S0 * (u ** j) * (d ** (n_steps - j))
    for j in range(n_steps + 1)
])
C_T = np.maximum(S_T - K, 0.0)  # call payoffs at maturity

# backward induction for the arbitrage-free option value
C = C_T.copy()
for step in range(n_steps, 0, -1):
    C = dt_disc * (q * C[1:step + 1] + (1.0 - q) * C[:step])

C0_tree = float(C[0])  # option value at time 0
print(f"binomial tree call value C0_tree = {C0_tree:.4f}")


### 6. Discounted price as a martingale

In a no-arbitrage model, discounted asset prices form a martingale under a risk-neutral
probability measure. In the binomial model this means that the current discounted price
equals the expected discounted price in the next period under the risk-neutral probability.

In [ ]:
disc = dt_disc  # per-step discount factor from the binomial tree

S_u = S0 * u  # stock level after an up move in one step
S_d = S0 * d  # stock level after a down move in one step

E_disc_S_next = disc * (q * S_u + (1.0 - q) * S_d)
print(f"discounted expected stock price = {E_disc_S_next:.4f}")
print(f"current stock price S0 = {S0:.4f}")


### 7. Link to continuous time

The binomial model approximates continuous-time models when the number of periods
increases and step sizes shrink. In the limit, a suitably scaled binomial model converges
to a diffusion model for the underlying, such as geometric Brownian motion, and a
continuous-time risk-neutral measure replaces the discrete risk-neutral probabilities.

In later chapters, these ideas lead to the Black--Scholes--Merton framework and to more
general models that still rely on the same principle: under a risk-neutral measure,
discounted asset prices are martingales and derivative prices are discounted expectations
of future payoffs.

<img src="https://hilpisch.com/tpq_logo_bic.png"
     width="30%"
     align="right"
     style="border-radius: 8px;">